Neste relÃ¡torio visamos treinar um algoritmo de regressÃ£o com base em 50.000 dados com 77 dimensÃµes usando a mÃ©trica MAE, erro absoluto mÃ©dio. Primeiramente diminuiremos as dimensÃµes dos dados com PCA, apÃ³s, treinaremos diversos regressores para, enfim, realizar um ensemble e reportar as saÃ­das esperadas.

In [133]:
# Includes
from   sklearn.gaussian_process import GaussianProcessRegressor
from   sklearn.model_selection  import GridSearchCV, StratifiedKFold, train_test_split
from   sklearn.neural_network   import MLPRegressor
from   sklearn.decomposition    import PCA
from   sklearn.preprocessing    import scale
from   sklearn.linear_model     import LinearRegression
from   sklearn.neighbors        import KNeighborsRegressor
from   sklearn.ensemble         import RandomForestRegressor, GradientBoostingRegressor
from   sklearn.metrics          import mean_absolute_error
from   sklearn.utils            import resample
from   sklearn.svm              import LinearSVR

import pandas as pd
import numpy  as np
import warnings
import math
import gc

#warnings.filterwarnings("ignore")

In [174]:
# LÃª os dados do arquivo
def readIt():
    from IPython.display import display
    df = pd.io.parsers.read_csv('ex6-train.csv')
    y = df.pop('V78').as_matrix()
    x = scale(df.as_matrix());
    display(df.head())
    return x, y

x, y = readIt()
data_size = len(x)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77
0,-0.325031,-0.311748,-0.31805,-0.315876,-0.317389,-0.327516,-0.324894,-0.360825,-0.340404,-0.343175,...,-0.071290,-0.095431,-0.090428,-0.325432,-0.312178,-0.318445,-0.316260,-0.317772,-0.327894,-0.325299
1,-0.320399,-0.314061,-0.31805,-0.315876,-0.318975,-0.329010,-0.320431,-0.350666,-0.340404,-0.343175,...,-0.782664,-0.793595,-0.090428,-0.320809,-0.314487,-0.318445,-0.316260,-0.319357,-0.329387,-0.320842
2,-0.322715,-0.316374,-0.31805,-0.315876,-0.318975,-0.323033,-0.324894,-0.355746,-0.345350,-0.343175,...,-0.782664,-0.095431,-0.090428,-0.323120,-0.316796,-0.318445,-0.316260,-0.319357,-0.323417,-0.325299
3,-0.322715,-0.316374,-0.31805,-0.314137,-0.318975,-0.329010,-0.324894,-0.355746,-0.345350,-0.343175,...,-0.782664,-0.793595,-0.090428,-0.323120,-0.316796,-0.318445,-0.314523,-0.319357,-0.329387,-0.325299
4,-0.325031,-0.314061,-0.31805,-0.315876,-0.317389,-0.326021,-0.321919,-0.360825,-0.340404,-0.343175,...,-0.071290,-0.095431,-0.090428,-0.325432,-0.314487,-0.318445,-0.316260,-0.317772,-0.326402,-0.322327


# Preprocessamento
## PCA

In [151]:
def pcaIt(x):
    pca = PCA(n_components=0.85)
    pca.fit(x)
    print("ApÃ³s o PCA temos", pca.n_components_,"componentes")
    return pca.transform(x)
    
x_reduced = pcaIt(x)

ApÃ³s o PCA temos 11 componentes


# AnÃ¡lise visual

In [ ]:
def plotIt():
    import matplotlib.pyplot as plt
    
    def plot(i, x, y, ncols, fig):
        if i % ncols == 0:
            fig = plt.figure(figsize=(15,2))
        ax = fig.add_subplot(1, ncols, 1+(i%ncols))
        ax.grid(color='lightgray', linestyle='--', linewidth=1)
        x_, y_ = [[*x] for x in zip(*sorted(zip(x,y)))]
        ax.plot(x_, y_, '.')
        ax.set_title("Componente {}".format(i+1))
        if i % ncols == ncols-1:
            plt.show()
        return fig

    transposed = np.transpose(x)
    transposed_reduced = np.transpose(x_reduced)
    
    fig = None
    ncols = 5
    print("Componentes do dado normalizado")
    for i in range(len(transposed)):
        fig = plot(i, transposed[i], y, ncols, fig)
    plt.show()

    print("Componentes do dado apÃ³s o PCA")
    for i in range(len(transposed_reduced[:])):
        fig = plot(i, transposed_reduced[i], y, ncols, fig)
    plt.show()
    
plotIt()

Analisar as componentes isoladamente pode gerar mÃ¡s interpretaÃ§Ãµes, mas podemos ver que algumas tÃªm comportamentos bem similares, outras, bem comportadas, e ainda hÃ¡ alguns casos, como nas componentes entre 43 e 49, onde os dados variam entre 2 valores.

# Treinamento

In [175]:
x = x_reduced
x_smallSample, y_smallSample = resample(x, y, n_samples=500)
gc.collect()

def mae_scorer(estimator, x, y):
    return mean_absolute_error(y, estimator.predict(x))

def testIt(regressor, name, params, resultStr, x, y):
    gscv = GridSearchCV(regressor, params, n_jobs=4, scoring='neg_mean_absolute_error', cv=5)
    gscv.fit(x,y)
    regressor = gscv.best_estimator_
    regressor.score = mae_scorer(regressor, x, y)
    paramList = [gscv.best_params_[name] for name in sorted(params.keys())]
    formatParams = [name, -gscv.best_score_, regressor.score] + paramList
    resultStr = "{} >> Menor MAE (Fold / Total): {:.3f} / {:.3f};\t" + resultStr
    print(resultStr.format(*formatParams))
    gc.collect()
    return regressor

## SVM

In [171]:
def trainSVM(x, y):
    regressor = LinearSVR(loss='epsilon_insensitive', random_state=0)
    params = {'epsilon':[0, 0.1, 1, 10], 'C':[2**-10, 2**-5, 2**1, 2**5]}
    resultStr = "C: {:.3f}; Eps: {}"
    return testIt(regressor, "SVM", params, resultStr, x, y)

trainSVM(x_smallSample, y_smallSample)

SVM >> Menor MAE (Fold / Total): 0.135 / 0.124;	C: 0.001; Eps: 0


LinearSVR(C=0.0009765625, dual=True, epsilon=0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=0, tol=0.0001, verbose=0)

## GBM

In [128]:
def trainGBM(x, y):
    regressor = GradientBoostingRegressor(loss='lad', random_state=0)
    params = {'learning_rate':[0.1, 0.05], 'max_depth':[3, 5, 7], 'n_estimators':[30, 70, 100]}
    resultStr = "L.R.: {}; Max Depth: {}; N Est.: {}"
    return testIt(regressor, "GBM", params, resultStr, x, y)

trainGBM(x_smallSample, y_smallSample)

Menor MAE (Fold / Total): 0.114 / 0.044;	L.R.: 0.1;	Max Depth: 7;	N Est.: 100


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='lad', max_depth=7, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=0, subsample=1.0, verbose=0,
             warm_start=False)

## RF

In [129]:
def trainRF(x, y):
    regressor = RandomForestRegressor(criterion='mae', random_state=0)
    params = {'n_estimators': [50, 100, 150], 'max_depth': [3, 5, 7]}
    resultStr = "Max Depth: {}; n_estimators: {}"
    return testIt(regressor, "RF", params, resultStr, x, y)

trainRF(x_smallSample, y_smallSample)

Menor MAE (Fold / Total): 0.117 / 0.082;	Max Depth: 3;	n_estimators: 50


RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=3,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=50, n_jobs=1, oob_score=False, random_state=0,
           verbose=0, warm_start=False)

## RN

In [130]:
def trainRN(x, y):
    regressor = MLPRegressor(random_state=0, alpha=1e+20)
    params = { 'hidden_layer_sizes':[(80,), (100,), (120,)], 'activation' : ['identity', 'logistic', 'tanh', 'relu']}
    resultStr = "hidden_layer_size: {}; activation: {}"
    return testIt(regressor, "RN", params, resultStr, x, y)

trainRN(x_smallSample, y_smallSample)

Menor MAE (Fold / Total): 0.385 / 0.384;	hidden_layer_size: logistic; 	activation: (120,)


MLPRegressor(activation='logistic', alpha=1e+20, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(120,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=0, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

## KNN

In [170]:
def trainKNN(x, y):
    regressor = KNeighborsRegressor(p=1)
    params = { 'n_neighbors':[1, 5, 11, 15, 21, 25], 'weights':['distance', 'uniform']}
    resultStr = "n_neighbors: {}; weights: {}"
    return testIt(regressor, "KNN", params, resultStr, x, y)

trainKNN(x_smallSample, y_smallSample)

KNN >> Menor MAE (Fold / Total): 0.212 / 0.000;	n_neighbors: 11; weights: distance


KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=11, p=1,
          weights='distance')

## Gaussian

In [132]:
def trainGaussian(x, y): 
    regressor = GaussianProcessRegressor(random_state=0)
    params = { 'alpha':[1e-9, 1e-6, 1e-3, 1]}
    resultStr = "alpha: {}"
    return testIt(regressor, "Gaussian", params, resultStr, x, y)

trainGaussian(x_smallSample, y_smallSample)

Menor MAE (Fold / Total): 0.214 / 0.128;	alpha: 1


GaussianProcessRegressor(alpha=1, copy_X_train=True, kernel=None,
             n_restarts_optimizer=0, normalize_y=False,
             optimizer='fmin_l_bfgs_b', random_state=0)

# Ensemble

In [172]:
# Os cÃ³digos para ensemble sÃ£o grandemente inspirados de uma resposta
# creditada Ã  Shahar Azulay & Ami Tavory. Acessado em 17/06/2017
# url: https://stackoverflow.com/a/35170149/3171285

# Tamanhos arbitrariamente escolhidos conforme capacidade da mÃ¡quina
sample_size = data_size*0.04
sample_size_hp = sample_size*0.8

# Inicializa o vetor de prediÃ§Ãµes para y
y_pred = np.zeros(n_regressors)

# Separa treino e teste para checar acurÃ¡cia da uniÃ£o dos 6 mÃ©todos
i_tr, i_ts = train_test_split(list(range(data_size)), train_size=sample_size, test_size=sample_size//4)
x_tr, x_ts, y_tr, y_ts = x[i_tr], x[i_ts], y[i_tr], y[i_ts]
    
# Subsample do treino para descobrir hiperparÃ¢metros
train_hp = resample(x_tr, y_tr, n_samples=sample_size_hp)

# Gera objetos dos regressores com busca de hiperparÃ¢metro
regressors = [trainSVM(*train_hp), 
              #trainGBM(*train_hp), 
              #trainRF(*train_hp), 
              #trainRN(*train_hp), 
              trainKNN(*train_hp), 
              #trainGaussian(*train_hp)
             ]
n_regressors = len(regressors)

# Gera a prediÃ§Ã£o para cada regressor
for i in range(n_regressors):
    # A principal diferenÃ§a entre estas duas linhas Ã© que a primeira tenta evitar overfitting atravÃ©s de treinos
    y_pred[i_tr, i] = Stacker(regressors[i]).fit_transform(x_tr, y_tr)[:,0] 
    y_pred[i_ts, i] = Stacker(regressors[i]).fit(x_ts, y_tr).transform(x_ts)
    
# Gera um regressor com base nas prediÃ§Ãµes
u = LinearRegression().fit(y_pred[i_tr, :], y_tr)

# Para cada re
for i in range(n_regressors):
    

# Calcula a pontuaÃ§Ã£o baseada
mae_scorer(u, y_pred[test_index, :], y[test_index])

SVM >> Menor MAE (Fold / Total): 0.096 / 0.095;	C: 0.031; Eps: 0
KNN >> Menor MAE (Fold / Total): 0.087 / 0.000;	n_neighbors: 5; weights: distance


Obs.:   O resultado esperado pelos dados de teste estÃ£o no arquivo ex6-result.csv
        SÃ£o criadas funÃ§Ãµes para trechos triviais para otimizar a memÃ³ria disponÃ­vel.

In [173]:

class Stacker(object):
    """
    A transformer applying fitting a predictor `pred` to data in a way
        that will allow a higher-up predictor to build a model utilizing both this 
        and other predictors correctly.

    The fit_transform(self, x, y) of this class will create a column matrix, whose 
        each row contains the prediction of `pred` fitted on other rows than this one. 
        This allows a higher-level predictor to correctly fit a model on this, and other
        column matrices obtained from other lower-level predictors.

    The fit(self, x, y) and transform(self, x_) methods, will fit `pred` on all 
        of `x`, and transform the output of `x_` (which is either `x` or not) using the fitted 
        `pred`.

    Arguments:    
        pred: A lower-level predictor to stack.

        cv_fn: Function taking `x`, and returning a cross-validation object. In `fit_transform`
            th train and test indices of the object will be iterated over. For each iteration, `pred` will
            be fitted to the `x` and `y` with rows corresponding to the
            train indices, and the test indices of the output will be obtained
            by predicting on the corresponding indices of `x`.
    """
    def __init__(self, pred, cv_fn=lambda x: sklearn.cross_validation.KFold().split(x)):
        self._pred, self._cv_fn  = pred, cv_fn

    def fit_transform(self, x, y):
        x_trans = self._train_transform(x, y)

        self.fit(x, y)

        return x_trans

    def fit(self, x, y):
        """
        Same signature as any sklearn transformer.
        """
        self._pred.fit(x, y)

        return self

    def transform(self, x):
        """
        Same signature as any sklearn transformer.
        """
        return self._test_transform(x)

    def _train_transform(self, x, y):
        x_trans = np.nan * np.ones((x.shape[0], 1))

        all_te = set()
        for tr, te in self._cv_fn(x):
            all_te = all_te | set(te)
            x_trans[te, 0] = self._pred.fit(x[tr, :], y[tr]).predict(x[te, :]) 
        if all_te != set(range(x.shape[0])):
            warnings.warn('Not all indices covered by Stacker', sklearn.exceptions.FitFailedWarning)

        return x_trans

    def _test_transform(self, x):
        return self._pred.predict(x)
    
    